In [1]:
import configargparse
import torch
import torch.optim as optim
from environments import CartPoleRegulatorEnv
from environments import CartEnv
from environments import AcrobotEnv
from models.agents import NFQAgent
from models.networks import NFQNetwork, ContrastiveNFQNetwork
from util import get_logger, close_logger, load_models, make_reproducible, save_models
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns

In [2]:
# Setup environment
bg_cart_mass = 1.0
fg_cart_mass = 1.0
init_experience = 400
force_left = 5
is_contrastive = False

train_env_bg = CartPoleRegulatorEnv(group=0, masscart=bg_cart_mass, mode="train", force_left=force_left, is_contrastive=is_contrastive)
train_env_fg = CartPoleRegulatorEnv(group=1, masscart=fg_cart_mass, mode="train", force_left=force_left, is_contrastive=is_contrastive)
eval_env_bg = CartPoleRegulatorEnv(group=0, masscart=bg_cart_mass, mode="eval", force_left=force_left, is_contrastive=is_contrastive)
eval_env_fg = CartPoleRegulatorEnv(group=1, masscart=fg_cart_mass, mode="eval", force_left=force_left, is_contrastive=is_contrastive)

# NFQ Main loop
bg_rollouts = []
fg_rollouts = []
total_cost = 0
if init_experience > 0:
    for _ in range(init_experience):
        rollout_bg, episode_cost = train_env_bg.generate_rollout(
            None, render=False, group=0
        )
        rollout_fg, episode_cost = train_env_fg.generate_rollout(
            None, render=False, group=1
        )
        bg_rollouts.extend(rollout_bg)
        fg_rollouts.extend(rollout_fg)
        total_cost += episode_cost

bg_rollouts.extend(fg_rollouts)
all_rollouts = bg_rollouts.copy()

bg_rollouts_test = []
fg_rollouts_test = []
if init_experience > 0:
    for _ in range(init_experience):
        rollout_bg, episode_cost = eval_env_bg.generate_rollout(
            None, render=False, group=0
        )
        rollout_fg, episode_cost = eval_env_fg.generate_rollout(
            None, render=False, group=1
        )
        bg_rollouts_test.extend(rollout_bg)
        fg_rollouts_test.extend(rollout_fg)
bg_rollouts_test.extend(fg_rollouts)
all_rollouts_test = bg_rollouts_test.copy()

/home/amandyam/.conda/envs/research/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
epoch = 3000
bg_success_queue = [0] * 3
fg_success_queue = [0] * 3
bg_converged = False

# Setup agent
nfq_net = ContrastiveNFQNetwork(state_dim=train_env_bg.state_dim, is_contrastive=is_contrastive)

optimizer = optim.Adam(nfq_net.parameters(), lr=1e-1)
nfq_agent = NFQAgent(nfq_net, optimizer)
nfq_agent._nfq_net.freeze_last_layers()
for ep in range(epoch + 1):
    
    if bg_converged:
        state_action_b, target_q_values, groups = nfq_agent.generate_pattern_set(fg_rollouts)
    else:
        state_action_b, target_q_values, groups = nfq_agent.generate_pattern_set(bg_rollouts)
    
    loss = nfq_agent.train((state_action_b, target_q_values, groups))
    
    eval_episode_length_fg = 0
    eval_episode_length_bg = 0
    eval_success_fg = False
    eval_success_bg = False
    if bg_converged:
        eval_episode_length_fg, eval_success_fg, eval_episode_cost_fg = nfq_agent.evaluate(
            eval_env_fg, render=False
        )
    else:
        eval_episode_length_bg, eval_success_bg, eval_episode_cost_bg = nfq_agent.evaluate(
        eval_env_bg, render=False
        )

    bg_success_queue = bg_success_queue[1:]
    bg_success_queue.append(1 if eval_success_bg else 0)

    fg_success_queue = fg_success_queue[1:]
    fg_success_queue.append(1 if eval_success_fg else 0)

    if sum(bg_success_queue) == 3:
        nfq_agent._nfq_net.unfreeze_last_layers()
        bg_converged = True

    if sum(fg_success_queue) == 3:
        print("FG Converged")
        break
    
    print("Epoch: " + str(ep) + " BG Converged: " + str(bg_converged) + " Eval BG: " + str(eval_episode_length_bg)
         + " Eval FG: " + str(eval_episode_length_fg))

Epoch: 0 BG Converged: False Eval BG: 24 Eval FG: 0
Epoch: 1 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 2 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 3 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 4 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 5 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 6 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 7 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 8 BG Converged: False Eval BG: 19 Eval FG: 0
Epoch: 9 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 10 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 11 BG Converged: False Eval BG: 28 Eval FG: 0
Epoch: 12 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 13 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 14 BG Converged: False Eval BG: 26 Eval FG: 0
Epoch: 15 BG Converged: False Eval BG: 20 Eval FG: 0
Epoch: 16 BG Converged: False Eval BG: 21 Eval FG: 0
Epoch: 17 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 18 BG Converged: False Eval BG: 21 Eval FG: 0
Epo

Epoch: 155 BG Converged: False Eval BG: 76 Eval FG: 0
Epoch: 156 BG Converged: False Eval BG: 61 Eval FG: 0
Epoch: 157 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 158 BG Converged: False Eval BG: 3000 Eval FG: 0
Epoch: 159 BG Converged: False Eval BG: 125 Eval FG: 0
Epoch: 160 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 161 BG Converged: False Eval BG: 50 Eval FG: 0
Epoch: 162 BG Converged: False Eval BG: 72 Eval FG: 0
Epoch: 163 BG Converged: False Eval BG: 107 Eval FG: 0
Epoch: 164 BG Converged: False Eval BG: 135 Eval FG: 0
Epoch: 165 BG Converged: False Eval BG: 25 Eval FG: 0
Epoch: 166 BG Converged: False Eval BG: 137 Eval FG: 0
Epoch: 167 BG Converged: False Eval BG: 77 Eval FG: 0
Epoch: 168 BG Converged: False Eval BG: 56 Eval FG: 0
Epoch: 169 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 170 BG Converged: False Eval BG: 63 Eval FG: 0
Epoch: 171 BG Converged: False Eval BG: 29 Eval FG: 0
Epoch: 172 BG Converged: False Eval BG: 52 Eval FG: 0
Epoch: 173 BG Converge

Epoch: 307 BG Converged: False Eval BG: 41 Eval FG: 0
Epoch: 308 BG Converged: False Eval BG: 41 Eval FG: 0
Epoch: 309 BG Converged: False Eval BG: 47 Eval FG: 0
Epoch: 310 BG Converged: False Eval BG: 57 Eval FG: 0
Epoch: 311 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 312 BG Converged: False Eval BG: 56 Eval FG: 0
Epoch: 313 BG Converged: False Eval BG: 23 Eval FG: 0
Epoch: 314 BG Converged: False Eval BG: 92 Eval FG: 0
Epoch: 315 BG Converged: False Eval BG: 74 Eval FG: 0
Epoch: 316 BG Converged: False Eval BG: 40 Eval FG: 0
Epoch: 317 BG Converged: False Eval BG: 47 Eval FG: 0
Epoch: 318 BG Converged: False Eval BG: 50 Eval FG: 0
Epoch: 319 BG Converged: False Eval BG: 47 Eval FG: 0
Epoch: 320 BG Converged: False Eval BG: 22 Eval FG: 0
Epoch: 321 BG Converged: False Eval BG: 49 Eval FG: 0
Epoch: 322 BG Converged: False Eval BG: 44 Eval FG: 0
Epoch: 323 BG Converged: False Eval BG: 57 Eval FG: 0
Epoch: 324 BG Converged: False Eval BG: 41 Eval FG: 0
Epoch: 325 BG Converged: Fal

Epoch: 459 BG Converged: False Eval BG: 37 Eval FG: 0
Epoch: 460 BG Converged: False Eval BG: 79 Eval FG: 0
Epoch: 461 BG Converged: False Eval BG: 99 Eval FG: 0
Epoch: 462 BG Converged: False Eval BG: 63 Eval FG: 0
Epoch: 463 BG Converged: False Eval BG: 3000 Eval FG: 0
Epoch: 464 BG Converged: False Eval BG: 75 Eval FG: 0
Epoch: 465 BG Converged: False Eval BG: 43 Eval FG: 0
Epoch: 466 BG Converged: False Eval BG: 56 Eval FG: 0
Epoch: 467 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 468 BG Converged: False Eval BG: 70 Eval FG: 0
Epoch: 469 BG Converged: False Eval BG: 56 Eval FG: 0
Epoch: 470 BG Converged: False Eval BG: 67 Eval FG: 0
Epoch: 471 BG Converged: False Eval BG: 45 Eval FG: 0
Epoch: 472 BG Converged: False Eval BG: 51 Eval FG: 0
Epoch: 473 BG Converged: False Eval BG: 107 Eval FG: 0
Epoch: 474 BG Converged: False Eval BG: 121 Eval FG: 0
Epoch: 475 BG Converged: False Eval BG: 62 Eval FG: 0
Epoch: 476 BG Converged: False Eval BG: 44 Eval FG: 0
Epoch: 477 BG Converged:

Epoch: 612 BG Converged: False Eval BG: 78 Eval FG: 0
Epoch: 613 BG Converged: False Eval BG: 88 Eval FG: 0
Epoch: 614 BG Converged: False Eval BG: 54 Eval FG: 0
Epoch: 615 BG Converged: False Eval BG: 64 Eval FG: 0
Epoch: 616 BG Converged: False Eval BG: 118 Eval FG: 0
Epoch: 617 BG Converged: False Eval BG: 68 Eval FG: 0
Epoch: 618 BG Converged: False Eval BG: 91 Eval FG: 0
Epoch: 619 BG Converged: False Eval BG: 62 Eval FG: 0
Epoch: 620 BG Converged: False Eval BG: 76 Eval FG: 0
Epoch: 621 BG Converged: False Eval BG: 66 Eval FG: 0
Epoch: 622 BG Converged: False Eval BG: 61 Eval FG: 0
Epoch: 623 BG Converged: False Eval BG: 84 Eval FG: 0
Epoch: 624 BG Converged: False Eval BG: 53 Eval FG: 0
Epoch: 625 BG Converged: False Eval BG: 67 Eval FG: 0
Epoch: 626 BG Converged: False Eval BG: 54 Eval FG: 0
Epoch: 627 BG Converged: False Eval BG: 68 Eval FG: 0
Epoch: 628 BG Converged: False Eval BG: 67 Eval FG: 0
Epoch: 629 BG Converged: False Eval BG: 163 Eval FG: 0
Epoch: 630 BG Converged: F

...

Epoch: 2269 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 2270 BG Converged: False Eval BG: 29 Eval FG: 0
Epoch: 2271 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2272 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2273 BG Converged: False Eval BG: 39 Eval FG: 0
Epoch: 2274 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 2275 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2276 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 2277 BG Converged: False Eval BG: 33 Eval FG: 0
Epoch: 2278 BG Converged: False Eval BG: 43 Eval FG: 0
Epoch: 2279 BG Converged: False Eval BG: 31 Eval FG: 0
Epoch: 2280 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 2281 BG Converged: False Eval BG: 41 Eval FG: 0
Epoch: 2282 BG Converged: False Eval BG: 38 Eval FG: 0
Epoch: 2283 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2284 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2285 BG Converged: False Eval BG: 33 Eval FG: 0
Epoch: 2286 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 228

Epoch: 2418 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 2419 BG Converged: False Eval BG: 44 Eval FG: 0
Epoch: 2420 BG Converged: False Eval BG: 45 Eval FG: 0
Epoch: 2421 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 2422 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2423 BG Converged: False Eval BG: 33 Eval FG: 0
Epoch: 2424 BG Converged: False Eval BG: 40 Eval FG: 0
Epoch: 2425 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2426 BG Converged: False Eval BG: 46 Eval FG: 0
Epoch: 2427 BG Converged: False Eval BG: 40 Eval FG: 0
Epoch: 2428 BG Converged: False Eval BG: 41 Eval FG: 0
Epoch: 2429 BG Converged: False Eval BG: 46 Eval FG: 0
Epoch: 2430 BG Converged: False Eval BG: 33 Eval FG: 0
Epoch: 2431 BG Converged: False Eval BG: 33 Eval FG: 0
Epoch: 2432 BG Converged: False Eval BG: 49 Eval FG: 0
Epoch: 2433 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2434 BG Converged: False Eval BG: 38 Eval FG: 0
Epoch: 2435 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 243

Epoch: 2568 BG Converged: False Eval BG: 50 Eval FG: 0
Epoch: 2569 BG Converged: False Eval BG: 43 Eval FG: 0
Epoch: 2570 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 2571 BG Converged: False Eval BG: 49 Eval FG: 0
Epoch: 2572 BG Converged: False Eval BG: 37 Eval FG: 0
Epoch: 2573 BG Converged: False Eval BG: 37 Eval FG: 0
Epoch: 2574 BG Converged: False Eval BG: 47 Eval FG: 0
Epoch: 2575 BG Converged: False Eval BG: 39 Eval FG: 0
Epoch: 2576 BG Converged: False Eval BG: 34 Eval FG: 0
Epoch: 2577 BG Converged: False Eval BG: 46 Eval FG: 0
Epoch: 2578 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2579 BG Converged: False Eval BG: 40 Eval FG: 0
Epoch: 2580 BG Converged: False Eval BG: 56 Eval FG: 0
Epoch: 2581 BG Converged: False Eval BG: 42 Eval FG: 0
Epoch: 2582 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 2583 BG Converged: False Eval BG: 34 Eval FG: 0
Epoch: 2584 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2585 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 258

Epoch: 2718 BG Converged: False Eval BG: 46 Eval FG: 0
Epoch: 2719 BG Converged: False Eval BG: 31 Eval FG: 0
Epoch: 2720 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2721 BG Converged: False Eval BG: 44 Eval FG: 0
Epoch: 2722 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2723 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2724 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2725 BG Converged: False Eval BG: 53 Eval FG: 0
Epoch: 2726 BG Converged: False Eval BG: 39 Eval FG: 0
Epoch: 2727 BG Converged: False Eval BG: 40 Eval FG: 0
Epoch: 2728 BG Converged: False Eval BG: 39 Eval FG: 0
Epoch: 2729 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2730 BG Converged: False Eval BG: 29 Eval FG: 0
Epoch: 2731 BG Converged: False Eval BG: 48 Eval FG: 0
Epoch: 2732 BG Converged: False Eval BG: 43 Eval FG: 0
Epoch: 2733 BG Converged: False Eval BG: 48 Eval FG: 0
Epoch: 2734 BG Converged: False Eval BG: 32 Eval FG: 0
Epoch: 2735 BG Converged: False Eval BG: 38 Eval FG: 0
Epoch: 273

Epoch: 2867 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2868 BG Converged: False Eval BG: 46 Eval FG: 0
Epoch: 2869 BG Converged: False Eval BG: 30 Eval FG: 0
Epoch: 2870 BG Converged: False Eval BG: 42 Eval FG: 0
Epoch: 2871 BG Converged: False Eval BG: 42 Eval FG: 0
Epoch: 2872 BG Converged: False Eval BG: 36 Eval FG: 0
Epoch: 2873 BG Converged: False Eval BG: 35 Eval FG: 0
Epoch: 2874 BG Converged: False Eval BG: 40 Eval FG: 0
Epoch: 2875 BG Converged: False Eval BG: 47 Eval FG: 0
Epoch: 2876 BG Converged: False Eval BG: 34 Eval FG: 0
Epoch: 2877 BG Converged: False Eval BG: 37 Eval FG: 0
Epoch: 2878 BG Converged: False Eval BG: 33 Eval FG: 0
Epoch: 2879 BG Converged: False Eval BG: 34 Eval FG: 0
Epoch: 2880 BG Converged: False Eval BG: 42 Eval FG: 0
Epoch: 2881 BG Converged: False Eval BG: 44 Eval FG: 0
Epoch: 2882 BG Converged: False Eval BG: 43 Eval FG: 0
Epoch: 2883 BG Converged: False Eval BG: 31 Eval FG: 0
Epoch: 2884 BG Converged: False Eval BG: 42 Eval FG: 0
Epoch: 288